# Training additional model (Part 3)

<img src="images\pen3.jpg" width="300" height="300">

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import pandas as pd
import numpy as np
import sqlite3
import os
import joblib

In [2]:
# Model parameters:
model_version = 2
model_id = int(model_version) + 100 
max_iter=30
random_state=42
#data_size = 165    We redefine this value later
scaled = True
method = 'LogisticRegression'

In [3]:
# Connect to database and add two new statuses
conn = sqlite3.connect('db_penguins.db')
cursor = conn.cursor()
conn.execute(f"INSERT INTO STATUS (status_id, status_type) VALUES ({model_version +1}, 'M{model_version}_train')")
conn.execute(f"INSERT INTO STATUS (status_id, status_type) VALUES ({model_version +2}, 'M{model_version +1}_test')")
conn.commit()


In [4]:
# Select all datapoints from the PENGUINS table that were not used in Model 1
query = f"SELECT * FROM PENGUINS WHERE status_id IS NULL"
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Data size is equal as the length of the df
data_size = len(df)

In [5]:
# Encode the 'species' column to numerical values
label_encoder = LabelEncoder()
df['species'] = label_encoder.fit_transform(df['species'])

# Selecting features and target variable
X = df[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g', 'animal_id']]
y = df['species']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# After splitting into X_train and X_test, get the 'animal_id' for each
train_ids = X_train['animal_id'].tolist()
test_ids = X_test['animal_id'].tolist()

In [6]:
# Convert train_ids and test_ids to a comma-separated string for storage in db table MODEL
fv_train = ','.join(map(str, train_ids))
fv_test = ','.join(map(str, test_ids))

In [7]:
# Update the database with the status for each animal_id
update_train_status = f"UPDATE PENGUINS SET status_id = {model_version +1} WHERE animal_id = ?"
update_test_status = f"UPDATE PENGUINS SET status_id = {model_version + 2} WHERE animal_id = ?"

conn = sqlite3.connect('db_penguins.db')
cursor = conn.cursor()

# Execute the updates
for aid in train_ids:
    conn.execute(update_train_status, (aid,))
for aid in test_ids:
    conn.execute(update_test_status, (aid,))

# Commit the changes and close the database connection
conn.commit()
conn.close()

In [8]:
# remove 'animal_id' from X_train and X_test
X_train = X_train.drop(['animal_id'], axis=1)
X_test = X_test.drop(['animal_id'], axis=1)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#X_train_scaled = X_train
#X_test_scaled = X_test

# Train a Logistic Regression Model
model = LogisticRegression(max_iter=max_iter, random_state=random_state)
model.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = model.predict(X_test_scaled)

# Evaluate the Model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 1.0
Confusion Matrix:
[[23  0  0]
 [ 0  9  0]
 [ 0  0 19]]


In [9]:
# Save model to local directory
model_directory = "/models"
model_filename = f"model_v{model_version}.joblib"
model_path = f".{model_directory}/{model_filename}"
joblib.dump(model, model_path)

['./models/model_v2.joblib']

In [10]:
# Write model data to sql
conn = sqlite3.connect('db_penguins.db')
cursor = conn.cursor()
# Insert model data into MODEL table including fv_train and fv_test
sql_insert_model = """
INSERT INTO MODEL (
    model_id,
    model_version,
    model_method,
    model_size,
    model_scaled,
    model_accuracy,
    fv_train,
    fv_test
) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""
cursor.execute(sql_insert_model, (model_id, model_version, method, data_size, int(scaled), accuracy, fv_train, fv_test))

conn.commit()
conn.close()
